In [27]:
import sys
sys.path.append("../")
import pandas as pd
import openpyxl as oxl

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Load data from the text file

In [28]:
# Read the file
with open('../data/raw/Incidents.txt') as f:
    lines = f.read().strip().split('\n')

incidents = []

header = [h.strip() for h in lines[0].split('|')]

for line in lines[1:]:
    if line.strip() == "":
        continue 
    values = [v.strip() for v in line.split('|')]
    entry = dict(zip(header, values))
    
    entry['ID'] = int(entry['ID'])
    entry['People'] = int(entry['People'])
    
    entry['Ages'] = [str(a.strip()) for a in entry['Ages'].split(',')]
    
    incidents.append(entry)

printdf = pd.DataFrame(incidents)
printdf


,ID,Time,Area,People,Ages,Main Need,Message
0,1,08:00 AM,Gampaha,4,[20-40],Water,"""Thirsty but safe on roof. Water level stable."""
1,2,08:15 AM,Ja-Ela,1,[75],Insulin,"""Diabetic, missed dose yesterday. Feeling faint."""
2,3,08:20 AM,Ragama,2,"[10, 35]",Rescue,"""Water approaching neck level. Child is crying."""


## Part A = Cot Scoring

### Cot Constraint

In [29]:
cot_instruction = """
You are a logistics decision-support system.

TASK:
Analyze the given incidents and assign a Priority Score using the rules below.

SCORING RULES (STRICT):
- Base score = 5
- +2 if any person is older than 60 OR younger than 5
- +3 if Main Need is "Rescue"
- +1 if Main Need is "Insulin"
- Maximum score = 10

OUTPUT FORMAT (MANDATORY):
Reasoning:
- Base score: 5
- Age factor: +X
- Need factor: +X
- Final score: Y/10

RULES:
- Do NOT invent new factors
- Do NOT change scoring weights
- Do NOT add extra text
- Follow format exactly
"""


### Cot LLM Client

In [30]:
model_cot = pick_model('groq', 'cot')
client_cot = LLMClient('groq', model_cot)

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='Logistic Decision Supporter',
    problem=incidents
)

### Get Cot output

In [ ]:
for idx, incident in enumerate(incidents, start=1): 
    full_prompt = f"{cot_instruction}\n\nIncident {idx} Details: {incident}"
    
    messages = [{"role": "user", "content": full_prompt}]
    
    response = client_cot.chat(messages, temperature=0.0, max_tokens=spec.max_tokens)
    
    incident["Analysis"] = response['text']
    
    print(f"Incident {incident['ID']}\n")
    print(response['text'])
    print("-" * 50)


Incident 1

Reasoning:
- Base score: 5
- Age factor: +0
- Need factor: +0
- Final score: 5/10
--------------------------------------------------
Incident 2

Reasoning:
- Base score: 5
- Age factor: +2
- Need factor: +1
- Final score: 8/10
--------------------------------------------------
Incident 3

Reasoning:
- Base score: 5
- Age factor: +0
- Need factor: +3
- Final score: 8/10
--------------------------------------------------


In [32]:
df = pd.DataFrame(incidents)
display(df)


,ID,Time,Area,People,Ages,Main Need,Message,Analysis
0,1,08:00 AM,Gampaha,4,[20-40],Water,"""Thirsty but safe on roof. Water level stable.""",Reasoning:\n- Base score: 5\n- Age factor: +0\...
1,2,08:15 AM,Ja-Ela,1,[75],Insulin,"""Diabetic, missed dose yesterday. Feeling faint.""",Reasoning:\n- Base score: 5\n- Age factor: +2\...
2,3,08:20 AM,Ragama,2,"[10, 35]",Rescue,"""Water approaching neck level. Child is crying.""",Reasoning:\n- Base score: 5\n- Age factor: +0\...


## Part B - Tot Strategy

### Tot Constraint

In [33]:
incident_list_text = "\n".join([
    f"ID{inc['ID']}: Area {inc['Area']}, Priority Score: {inc['Analysis'].split('Final score: ')[1].strip()} - ({inc['Analysis']})"
    for inc in incidents
])


tot_instruction = f"""
You are a Logistics Commander. You have ONE rescue boat at Ragama.
Your task is to plan the optimal sequence to visit the incidents to maximize total priority score.

Below are the incidents with their priority scores and short explanations. Do NOT change these scores or explanations:
{incident_list_text}

Rules and considerations:
- Only ONE incident can be handled per stop.
- Travel times between areas (in minutes):
  - Ragama -> Ja-Ela: 10
  - Ja-Ela -> Gampaha: 40
  - Ragama -> Gampaha: 50
  Symmetric travel times assumed.

Explore 3 distinct branches:
1. Branch 1 (Greedy): Save highest score first.
2. Branch 2 (Speed): Save closest first.
3. Branch 3 (Logistics): Save furthest first.

Instructions:
- For each branch, summarize in natural language how the incidents are visited.
- Include the sequence of incident IDs, total priority score, and total travel time.
- After all branches, provide a brief comparison highlighting trade-offs between total score and travel time.
- Conclude with your recommended optimal route and reasoning.
- Use only the incident IDs and priority scores provided.
- Present the output clearly and readably without a rigid table format.
"""

### Tot LLM Client

In [ ]:
model_tot = pick_model('groq', 'tot')
client_tot = LLMClient('groq', model_tot)

prompt_text, spec = render(
    'tot_reasoning.v1',
    role='Logistic Commander',
    problem=incidents, 
    branches='3'
)

### Get Tot output

In [ ]:
full_prompt = tot_instruction  

messages = [{'role': 'user', 'content': full_prompt}]

response = client_tot.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

display(Markdown('ToT Response (Multiple Solution Paths):'))
display(Markdown('=' * 80))
display(Markdown(response['text']))
display(Markdown('=' * 80))

ToT Response (Multiple Solution Paths):

================================================================================

To determine the optimal sequence for visiting the incidents, we will explore three distinct branches: Greedy, Speed, and Logistics.

**Branch 1: Greedy**
In this approach, we prioritize the incidents based on their priority scores, starting with the highest. The sequence of incident IDs is ID2 (8/10) and ID3 (8/10) which are tied, then ID1 (5/10). To break the tie, we can choose either ID2 or ID3 first since they have the same score and are equidistant from the starting point in terms of priority. Let's start with ID3 since it's at Ragama, where the boat is initially located, saving travel time for the first leg. The sequence becomes ID3, ID2, ID1. 
- Sequence: ID3, ID2, ID1
- Total priority score: 8 + 8 + 5 = 21
- Total travel time: 0 (starting at Ragama) + 10 (Ragama to Ja-Ela) + 40 (Ja-Ela to Gampaha) = 50 minutes

**Branch 2: Speed**
Here, we prioritize incidents based on their proximity to the current location, aiming to minimize travel time. Starting from Ragama, the closest incident is ID3 (at Ragama), then ID2 (Ja-Ela), and finally ID1 (Gampaha).
- Sequence: ID3, ID2, ID1
- Total priority score: 8 + 8 + 5 = 21
- Total travel time: 0 (starting at Ragama) + 10 (Ragama to Ja-Ela) + 40 (Ja-Ela to Gampaha) = 50 minutes

**Branch 3: Logistics**
In this approach, we visit the furthest incident first to get it out of the way, potentially optimizing the overall route efficiency. Starting from Ragama, the furthest incident is ID1 (Gampaha), then ID2 (Ja-Ela), and ID3 (Ragama).
- Sequence: ID1, ID2, ID3
- Total priority score: 5 + 8 + 8 = 21
- Total travel time: 50 (Ragama to Gampaha) + 40 (Gampaha to Ja-Ela) + 10 (Ja-Ela to Ragama) = 100 minutes

**Comparison and Conclusion**
All three branches yield the same total priority score of 21. However, they differ significantly in total travel time: Branch 1 (Greedy) and Branch 2 (Speed) both result in 50 minutes of travel time, while Branch 3 (Logistics) requires 100 minutes. The trade-off here is between the efficiency of the route (travel time) and the priority of incidents. Since both the Greedy and Speed approaches yield the same sequence and travel time, and given that they both maximize the priority score while minimizing travel time compared to the Logistics approach, either could be considered optimal depending on the specific goals of the mission. However, given the constraints and the goal to maximize the total priority score while considering travel time, the sequence ID3, ID2, ID1 (starting with the highest priority incident that is also the closest to the starting point) is the most efficient and effective approach.

**Recommended Optimal Route**
The recommended sequence is ID3, ID2, ID1, which combines the benefits of starting with a high-priority incident (ID3) that is at the starting location, followed by another high-priority incident (ID2) that is close, and finally addressing the lower-priority but still important incident (ID1). This approach balances the need to assist those in need quickly with the practical consideration of travel time, thereby optimizing the use of the single rescue boat.

================================================================================